In [ ]:
df = pd.read_csv('/kaggle/input/spotify-dataset-scaled/df_scaled.csv')

In [ ]:
df_full = pd.read_csv('/kaggle/input/spotify-songs/spotify_songs.csv')

In [ ]:
df.head()

# Training

In [ ]:
# Librerías necesarias
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Selecting relevant audio features
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
            'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'year']
#Drop: 'genre_edm', 'genre_latin', 'genre_pop', 'genre_r&b', 'genre_rap', 'genre_rock'
#Reintroducir features: mode?

X = df[features].values

In [ ]:
X = X.astype('float32')

In [ ]:
# Splitting the data
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)

In [ ]:
model = Sequential([
    Input(shape=(len(features),)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu', name="embedding_layer"), # Embedding layer
    Dense(16, activation='relu'),
    Dense(len(features), activation='linear') # Output layer for reconstruction
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Training the model
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np

# Define model with the functional API
input_layer = Input(shape=(len(features),))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.3)(x)
embedding_layer = Dense(64, activation='relu', name="embedding_layer")(x)  # Named embedding layer
x = Dropout(0.3)(embedding_layer)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(features), activation='linear')(x)

# Compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Initialize the model by running a sample input through it
sample_input = np.random.rand(1, len(features))  # Sample input
_ = model.predict(sample_input)  # Initialize the model

# Extract embeddings using the embedding layer directly
embedding_model = Model(inputs=model.input, outputs=model.get_layer("embedding_layer").output)
song_embeddings = embedding_model.predict(X)

print("Embeddings shape:", song_embeddings.shape)


# Recommendation

In [8]:
# Búsqueda por género
not_pop = df[df['genre_latin'] == True]
not_pop

,track_id,track_popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,year,genre_edm,genre_latin,genre_pop,genre_r&b,genre_rap,genre_rock
12407,510UgVCpnUaQ5ACHW2J8gu,0.550562,0.514089,0.549081,0.410433,0,0.051643,0.144100,0.000000,0.124869,0.188,0.373357,0.225441,0.953133,False,True,False,False,False,False
12408,7oxHEN7q9V67XUReaq0txr,0.438202,0.540225,0.342225,0.268287,1,0.046406,0.113329,0.000076,0.070458,0.130,0.464684,0.334233,0.863838,False,True,False,False,False,False
12409,3876zu2BcDL39vLkkJGKLZ,0.370787,0.372100,0.559504,0.399200,0,0.046406,0.071017,0.000000,0.106160,0.624,0.334313,0.222089,0.953133,False,True,False,False,False,False
12410,4w3InOZmItJiSlhnkr3lv6,0.629213,0.455625,0.683929,0.518202,0,0.037296,0.054393,0.000002,0.118672,0.446,0.503663,0.208276,0.953133,False,True,False,False,False,False
12411,12f3UbBnYHrZ7vbRXfyDA9,0.505618,0.519841,0.258064,0.287549,0,0.077794,0.055529,0.011533,0.103459,0.181,0.412193,0.202602,0.953133,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15793,75wtugYpepfnAzjVH6twz4,0.280899,0.390625,0.697225,0.464556,0,0.045833,0.279146,0.000000,0.066256,0.849,0.790185,0.302944,0.702332,False,True,False,False,False,False
15794,3lohME7pf8CIbVoToTdWV5,0.292135,0.414736,0.700569,0.403222,0,0.053730,0.190620,0.000000,0.248421,0.806,0.217161,0.310435,0.702332,False,True,False,False,False,False
15795,2Zt4IqQqqVbIasXtNvJMDZ,0.292135,0.444889,0.705600,0.480730,0,0.048123,0.123986,0.000000,0.160417,0.882,0.816486,0.234757,0.821362,False,True,False,False,False,False
15796,3jmBWNWrQErNPBPiE0friv,0.146067,0.452929,0.893025,0.495140,1,0.042101,0.331460,0.000043,0.418052,0.803,0.282128,0.254472,0.821362,False,True,False,False,False,False


In [1]:
# Búsqueda por artista/nombre
df_full.loc[df_full['track_artist'] == 'Daft Punk']
#df_full.loc[df_full['track_name'] == 'Yesterday']

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
589,5CMjjywI0eZMixPeqNd75R,Lose Yourself to Dance,Daft Punk,71,4m2880jivSbbyEGAKfITCa,Random Access Memories,2013-05-17,Dance Pop Tunes,4SdfG4cPG3skmTiQLozZGh,pop,...,10,-7.828,0,0.0570,0.083900,0.001140,0.0753,0.674,100.163,353893
604,2Foc5Q5nqNiosCNqttzHof,Get Lucky (feat. Pharrell Williams & Nile Rodg...,Daft Punk,76,2ePFIvZKMe8zefATp9ofFA,Get Lucky (feat. Pharrell Williams & Nile Rodg...,2013-04-19,Dance Pop Tunes,4SdfG4cPG3skmTiQLozZGh,pop,...,6,-8.966,0,0.0380,0.042600,0.000001,0.1010,0.862,116.047,248413
1223,2cGxRwrMyEAp8dEbuZaVv6,Instant Crush,Daft Punk,75,4m2880jivSbbyEGAKfITCa,Random Access Memories,2013-05-17,Pop Inglés (2020 - 2010s)💙 Música En Inglés 2010s,4QAzO1Z92yqqD6Mf3mOANu,pop,...,10,-9.516,0,0.0271,0.042200,0.619000,0.0770,0.518,109.942,337560
2679,2cGxRwrMyEAp8dEbuZaVv6,Instant Crush,Daft Punk,75,4m2880jivSbbyEGAKfITCa,Random Access Memories,2013-05-17,ELECTROPOP,2UsEj2GUukV0GLbsE3rldz,pop,...,10,-9.516,0,0.0271,0.042200,0.619000,0.0770,0.518,109.942,337560
2681,0DiWol3AO6WpXZgp0goxAV,One More Time,Daft Punk,74,2noRn2Aes5aoNVsU6iWThc,Discovery,2001-03-07,ELECTROPOP,2UsEj2GUukV0GLbsE3rldz,pop,...,2,-8.618,1,0.1330,0.019300,0.000000,0.3320,0.476,122.752,320357
3031,2LD2gT7gwAurzdQDQtILds,Veridis Quo,Daft Punk,62,2noRn2Aes5aoNVsU6iWThc,Discovery,2001-03-07,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,pop,...,9,-13.928,0,0.0450,0.479000,0.950000,0.1040,0.119,106.896,345187
3032,1pKYYY0dkg23sQQXi0Q5zN,Around the World,Daft Punk,70,5uRdvUR7xCnHmUW8n64n9y,Homework,1997-01-16,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,pop,...,7,-5.311,1,0.1500,0.003560,0.889000,0.0906,0.841,121.294,429533
3033,5CMjjywI0eZMixPeqNd75R,Lose Yourself to Dance,Daft Punk,71,4m2880jivSbbyEGAKfITCa,Random Access Memories,2013-05-17,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,pop,...,10,-7.828,0,0.0570,0.083900,0.001140,0.0753,0.674,100.163,353893
3034,2Foc5Q5nqNiosCNqttzHof,Get Lucky (feat. Pharrell Williams & Nile Rodg...,Daft Punk,76,2ePFIvZKMe8zefATp9ofFA,Get Lucky (feat. Pharrell Williams & Nile Rodg...,2013-04-19,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,pop,...,6,-8.966,0,0.0380,0.042600,0.000001,0.1010,0.862,116.047,248413
3035,69kOkLUCkxIZYexIgSG8rq,Get Lucky,Daft Punk,73,4m2880jivSbbyEGAKfITCa,Random Access Memories,2013-05-17,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,pop,...,6,-9.404,0,0.0403,0.037800,0.000002,0.0720,0.863,116.049,369627


Reggaeton: 3gu0fRSgFuc4FmrtIv0DnC

In [2]:
# Búsqueda por ID
id_ = "5CMjjywI0eZMixPeqNd75R"
name = df_full.loc[df_full['track_id'] == id_]
print(name.iloc[0]['track_name'], 'by', name.iloc[0]['track_artist'], 'on album', name.iloc[0]['track_album_name'])
song_index = df[df['track_id'] == id_].index.item()
print(song_index)
print()

Lose Yourself to Dance by Daft Punk on album Random Access Memories
559



In [3]:
# Búsqueda por index
#song_index = 12616  # For example, the first song in the dataset

name = df_full.loc[df_full['track_id'] == df.iloc[song_index]['track_id']]
print(name.iloc[0]['track_name'], 'by', name.iloc[0]['track_artist'], 'on album', name.iloc[0]['track_album_name'])
print()

df.iloc[song_index]

Lose Yourself to Dance by Daft Punk on album Random Access Memories



track_id            5CMjjywI0eZMixPeqNd75R
track_popularity                  0.674157
danceability                      0.692224
energy                            0.434281
loudness                          0.350094
mode                                     0
speechiness                       0.055435
acousticness                      0.080566
instrumentalness                  0.001139
liveness                            0.0726
valence                              0.674
tempo                              0.33533
duration_ms                       0.489098
year                              0.702332
genre_edm                            False
genre_latin                          False
genre_pop                             True
genre_r&b                            False
genre_rap                            False
genre_rock                           False
Name: 559, dtype: object

id 1: 0r7CVbZTWZgbTCYdfa2P31 (pop)

id 4445: 5yY9lUy8nbvjM1Uyo1Uqoc (rap)

In [4]:
similarities = cosine_similarity([song_embeddings[song_index]], song_embeddings)[0]

# Get top 5 most similar songs (excluding itself)
similar_indices = np.argsort(similarities)[-6:-1][::-1]
similar_songs = df.iloc[similar_indices]

In [5]:
for song in range(len(similar_songs)):
    name = df_full.loc[df_full['track_id'] == similar_songs.iloc[song]['track_id']]
    print(name.iloc[0]['track_name'], 'by', name.iloc[0]['track_artist'], 'on album', name.iloc[0]['track_album_name'])
    print()

Nobody Better - Blacksmith R'n'B Rub by Tina Moore on album The Legacy of Rn'B Party

MD - Radio Edit by MZ on album La dictature

Salió El Sol - Remastered 2016 by Don Omar on album King Of Kings 10th Anniversary (Remastered)

Enjoy Your Life - Edit by Rayko on album Enjoy Your Life

Can't You See by Rick Wade on album Players Theme



In [6]:
similar_songs

,track_id,track_popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,year,genre_edm,genre_latin,genre_pop,genre_r&b,genre_rap,genre_rock
12903,44ETcHYUz11q8QbqcgSwL6,0.561798,0.534361,0.685584,0.593168,0,0.087461,0.015184,0.000000,0.340749,0.704,0.341249,0.205894,0.953133,False,True,False,False,False,False
17347,14lgKkkbNsJl9fItqnEfpV,0.651685,0.521284,0.595984,0.462749,1,0.044782,0.143234,0.000000,0.290428,0.687,0.347363,0.243979,0.953133,False,False,False,True,False,False
17421,57kuq3bJCohw5rUtT5H2gN,0.325843,0.559504,0.688900,0.530601,0,0.049076,0.026642,0.000005,0.275356,0.779,0.340903,0.231615,0.953133,False,False,False,True,False,False
13734,51uLQESb4VnYAbs5WtGxho,0.696629,0.537289,0.611524,0.518515,1,0.037874,0.144966,0.000000,0.251537,0.634,0.288777,0.290716,0.953133,False,True,False,False,False,False
19786,3U548qE3Pb8R1jdjWotoBz,0.314607,0.521284,0.710649,0.386106,1,0.070272,0.042868,0.003793,0.297137,0.738,0.489975,0.231918,0.953133,True,False,False,False,False,False


In [7]:
import numpy as np
import pandas as pd

# 0. Canción original (user input)
song = df.iloc[song_index].copy()
song = song.drop(index=["track_id", "track_popularity","mode"])
song = song.astype(float)

# 1. Calcular la media de las características en el dataframe completo
df_mean = df[features].mean()

# 2. Filtrar las canciones similares usando los índices y calcular la media
similar_mean = similar_songs[features].mean()

# 3. Crear un DataFrame que contenga ambas medias para fácil comparación
comparison_df = pd.DataFrame({'Original': song, 'Similar songs': similar_mean})

# 4. Calcular el RMSE entre la canción original y la media de las canciones similares
comparison_df['Similarity'] = np.sqrt((comparison_df['Original'] - comparison_df['Similar songs']) ** 2)
comparison_df['Other songs'] = df_mean


# 5. Calcular el RMSE entre la media de canciones similares y la media general
comparison_df['Difference'] = np.sqrt((comparison_df['Similar songs'] - comparison_df['Other songs']) ** 2)


# Mostrar el DataFrame de comparación
print(f"Comparison of Features between Overall Mean and Similar Songs Mean:\n")
print(comparison_df)


Comparison of Features between Overall Mean and Similar Songs Mean:

                  Original  Similar songs  Similarity  Other songs  Difference
acousticness      0.069899       0.074579    0.004680     0.152717    0.078139
danceability      0.527076       0.534744    0.007668     0.451754    0.082991
duration_ms       0.182352       0.240824    0.058472     0.285104    0.044280
energy            0.664225       0.658528    0.005697     0.513626    0.144902
genre_edm         0.000000            NaN         NaN          NaN         NaN
genre_latin       0.000000            NaN         NaN          NaN         NaN
genre_pop         1.000000            NaN         NaN          NaN         NaN
genre_r&b         0.000000            NaN         NaN          NaN         NaN
genre_rap         0.000000            NaN         NaN          NaN         NaN
genre_rock        0.000000            NaN         NaN          NaN         NaN
instrumentalness  0.004201       0.000760    0.003442     0.06